# Visualization: Trading Session

In [4]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [6]:
from trading_bot.agent import Agent

model_name = 'ddqn_EU_50'
test_stock = 'data/EUR_USD_TEST.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

AttributeError: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

### 2. Load test data

In [3]:
# read csv into dataframe
df = pd.read_csv(test_stock)

# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names

df = df.rename(columns={'Adj Close': 'Closing Price'})
# convert dates from object to DateTime type

dates = df['Date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['Date'] = dates

df.head()

Date  Closing Price
0 2020-01-01         1.1212
1 2020-01-02         1.1172
2 2020-01-03         1.1160
3 2020-01-06         1.1195
4 2020-01-07         1.1153

### 3. Running Eval

In [4]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG switching to TensorFlow for CPU
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Buy at: $1.12
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Sell at: $1.12 | Position: -$0.0040
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Buy at: $1.12
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Sell at: $1.11 | Position: -$0.0048
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Buy at: $1.11
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Buy at: $1.10
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Sell at: $1.09 | Position: -$0.0203
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Buy at: $1.10
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Sell at: $1.10 | Position: +$0.0004
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] DEBUG Sell at: $1.11 | Position: +$0.0134
2020-03-28 16:29:13 DESKTOP-UBSIEG2 root[11828] INFO ddqn_EU_50: -$0.0153



### 4. Visualize the Double DQ Learning

In [7]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    df['size'] = "Buy/Sell"
   
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['Closing Price']), min(df['position'])), max(max(df['Closing Price']), max(df['position']))), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='Date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('Date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action',size = 'size'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=800)
    
    return chart

In [8]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)